# Project overview
## What are you doing?
I'm trying to implement some common deep learning architectures from the ground up, starting with a simple feedforward neural net and hopefully moving on to some more interesting structures like long short-term memory models. By "from the ground up", I mean using as few existing libraries as possible and writing my own code without reference to prior implementations. I'm also documenting this process as I go.
## Ok but why are you doing that
I'm trying to get a better handle on the fundamentals before I start in on projects using more complicated architectures. I know how a neural net operates at a fairly high level, but I don't use anything ML-related in my day job and it's not something I focused on much in college. I want to prove to myself that I really understand how, say, backprogation works during the training process, in hopes that this will help me develop better intuitions about more complex structures. I'm not aiming for some highly efficient deep learning library here; I just want a toy net that can classify iris species or whatever.

The documentation part is mainly for my own benefit, so that I have a record of what I've figured out and what I want to try next. However, if you're reading this and you're not me, cool! I'm planning on writing this as if for a (small, informal) audience, and maybe it'll be helpful to someone else down the road. Caveat: I'm not an authoritative source on any of this and I will probably make some misleading/fully incorrect statements as I figure things out, so please take things with a grain of salt and double-check any math I'm putting in here. I'll link sources that I find helpful as I go.
## What's the plan?
Based on my limited research so far, I'm thinking something like feedforward nets -> simple RNNs -> LSTMs -> transformers, with various detours along the way depending on what I find out. I'm mainly interested in language rather than other applications, so CNNs are off the list for now. I haven't heard a ton about GANs for language applications either.

I don't expect that any of my implementations of architectures beyond simple feedforward will be able to handle any real training in a way that is efficient enough for actual use, so for any projects using those architectures I'll use something that actually works - I'll build my own slow, shitty version to understand what's happening, then use the pytorch version in an application.
## What's the end goal?
To know more? I guess? Ideally this will give me a better foundation for larger projects. I've been kicking around the idea of applying a generative language architecture to a corpus of music scores and trying out AI-assisted composition, but right now I don't really have enough know-how to set that up properly, and I don't want to just follow some out-of-the-box model. I'd also like to be more informed about new developments in AI, which are happening at an alarming rate these days.
## What level of knowledge are you starting with, exactly?
CS degree, decent math background, and 3 1/2 years of non-ML-related industry experience. The CS degree included some courses that collectively spent maybe a week on neural nets (including calculating a bunch of training example gradients by hand) and several more weeks on other ML topics (from linear regression to support vector machines). I was very linguistics-focused in undergrad, so I also spent quite a bit of time on practical NLP and traditional/computational linguistics. A lot of this is out of date, though, so I anticipate relearning most of this to catch up with state of the art. I'm essentially a data engineer in my day job, and I have a good amount of experience with big data processing, although not at the scale that modern models would need.
## Sounds good! Get started!
yeah yeah yeah

# 1.a Feedforward neural net structure
Things I remember about how neural networks are set up:
* One net is composed of two or more layers, each of which is composed of some number of neurons
* Each neuron in one layer is connected to all the neurons in the next layer
* Each connection has a weight
* Each neuron has a particular activation function
So let's start at the bottom with a Neuron object. (Yes, I know that, in practice, a neural net is basically just an alternating chain of matrix multiplications and elementwise vector operations. Humor me while I make the nominal structure extremely explicit.)

In [3]:
class Neuron(object):
    
    # Initialize with an activation function
    # Weights and links will be determined later
    def __init__(self, f):
        self.activation = f
        self.upstream_neurons = []
        self.upstream_weights = []
        self.downstream_neurons = []
        self.downstream_weights = []
        self.current_value = None

    # Link from an earlier neuron
    # Input array and weight array are paired
    def link_input(self, input_neuron, weight):
        self.upstream_neurons.append(input_neuron)
        self.upstream_weights.append(weight)
        input_neuron.downstream_neurons.append(self)
        input_neuron.downstream_weights.append(weight)

    # Link to a later neuron
    # Should use one or the other of the link functions for any given net
    # to avoid double binding neurons
    def link_output(self, output_neuron, weight):
        self.downstream_neurons.append(output_neuron)
        self.downstream_weights.append(weight)
        output_neuron.upstream_neurons.append(self)
        output_neuron.upstream_weights.append(weight)
        

I need an activation function to instantiate this and I don't remember the formulas for any of the standard ones, but fortunately [this TDS article](https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6) has a good table of popular activations (plus their derivatives, which I know are going to be useful for training later). I'm being pretty traditional here, so let's use sigmoid.

In [5]:
import math

def sigmoid(x):
    return 1 / (1 + math.e ** (-x))

So I've already had to do my first import, but in my defense it's only so I can get a good value of *e* without hardcoding it somewhere. Anyway though now we can instantiate a neuron.

In [8]:
n = Neuron(sigmoid)
n.activation(0)

0.5

Cool. I think a "Layer" class would be overengineering here (says the one who's building a bad neural net from  scratch) since there's not much additional structure that isn't better captured in either a neuron or in a complete net. There's maybe cause to introduce a few neuron subclasses (e.g. SigmoidNeuron, TanhNeuron) with set activations, but let's do that later if it seems useful. Moving on to the Net class, which is going to be defined as a set of layers, each of which has a size and an activation function (maybe some architectures I don't know about use a variety of activations within a layer, but we'll ignore that for now).